In [31]:
!pip install nilearn==0.9.2

# !pip install keras_cv


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
import os
from sklearn.linear_model import LinearRegression
import tensorflow_datasets as tfds
import tensorflow as tf
from keras import datasets, Sequential
from keras.layers import Conv2D, Dense, MaxPooling2D, Flatten
import numpy as np
from sklearn.metrics import confusion_matrix
# from tensorflow.keras.layers.experimental.preprocessing import Resizing
# import keras_cv
import keras
import datetime

# %load_ext tensorboard

# Load the data

In [33]:
########## LOADING CIFAR DATASET #############

cifar, cifar_info = tfds.load("cifar100", as_supervised=True, with_info =True)

# split data into train and test
train_ds = cifar["train"]
val_ds = cifar["test"]


# Preprocess data

In [34]:
############# PREPROCESS FUNCTION #################

def data_pip(cifar10, dataset_type, batch_size):

  #cifar10 = tf.data.Dataset.from_tensor_slices(cifar10)
  cifar10 = cifar10.map(lambda img, target: (tf.cast(img, tf.float32), target))
  #sloppy input normalization, just bringing image values from range [0, 255] to [-1, 1]
  cifar10 = cifar10.map(lambda img, target: ((img/128.)-1., target))
  #create one-hot targets
  cifar10 = cifar10.map(lambda img, target: (img, tf.one_hot(target, depth=100)))
  #cache this progress in memory, as there is no need to redo it; it is deterministic after all
  # cifar10 = cifar10.cache()
  # shuffle, batch, prefetch
  cifar10 = cifar10.shuffle(1000)
  # cifar10 = cifar10.map(lambda img, target: (tf.expand_dims(img, 0), target))
  cifar10 = cifar10.batch(64)
  cifar10 = cifar10.prefetch(20)
  return cifar10

In [ ]:
print(type(train_ds))

<class 'tensorflow.python.data.ops.dataset_ops.PrefetchDataset'>


In [35]:
train_cifar = data_pip(train_ds, 'cifar', 32)
test_cifar = data_pip(val_ds, 'cifar', 32)

In [ ]:
########### PREPROCESS ################

#train_cifar = tf.data.Dataset.from_tensor_slices((train_ds))
#test_cifar = tf.data.Dataset.from_tensor_slices((val_ds)

'''
issa = tf.data.Dataset.from_tensor_slices((train_fab, train_y))

cifar_data = data_pip(issa, 'cifar', 32)
print(cifar_data) '''

# Visualization

In [ ]:
############ VISUALIZATION ################

# tfds.show_examples(cifar_data, cifar_info)
# tfds.show_examples ( train_ds , ds_info )

In [ ]:
    model.add(Conv2D(input_shape=(32, 32, 3), kernel_size=(2, 2), padding='same', strides=(2, 2), filters=32))
    model.add(Conv2D(input_shape=(32, 32, 3), kernel_size=(2, 2), padding='same', strides=(2, 2), filters=32))
    model.add(Conv2D(input_shape=(32, 32, 3), kernel_size=(2, 2), padding='same', strides=(2, 2), filters=32))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='same'))

    model.add(Conv2D(kernel_size=(2, 2), padding='same', strides=(2, 2), filters=64))
    model.add(Conv2D(kernel_size=(2, 2), padding='same', strides=(2, 2), filters=64))
    model.add(Conv2D(kernel_size=(2, 2), padding='same', strides=(2, 2), filters=64))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='same'))
    
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(100, activation='softmax'))


[tf.keras.layers.Conv2D(input_shape=(32, 32, 3), kernel_size=(2, 2), padding='same', strides=(2, 2), filters=32),
                           tf.keras.layers.Conv2D(input_shape=(32, 32, 3), kernel_size=(2, 2), padding='same', strides=(2, 2), filters=32),
                           tf.keras.layers.Conv2D(input_shape=(32, 32, 3), kernel_size=(2, 2), padding='same', strides=(2, 2), filters=32),
                           tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='same'),
                           tf.keras.layers.Conv2D(kernel_size=(2, 2), padding='same', strides=(2, 2), filters=64),
                           tf.keras.layers.Conv2D(kernel_size=(2, 2), padding='same', strides=(2, 2), filters=64),
                           tf.keras.layers.Conv2D(kernel_size=(2, 2), padding='same', strides=(2, 2), filters=64),
                           tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='same')),
                           tf.keras.layers.Flatten(),
                           tf.keras.layers.Dense(256, activation='relu'),
                           tf.keras.layers.Dense(128, activation='relu'),
                           tf.keras.layers.Dense(100, activation='softmax')]


In [38]:
######## NEUES MODEL TRAYYYY ##########

#from tensorflow.python.framework.importer import import_graph_def_for_function
class BasicConv(tf.keras.Model):
    def __init__(self, L2_reg=0, dropout_rate=0, batch_norm=False): # penalties, dropout, batch normalization

        """ 
        subclass of the tf.keras.Model class, creates metrics
        Args:
            L2_reg(float) = regularizer that applies a L2 regularization penalty
            dropout_rate(float) = sets input units to 0 with a frequency dropout_rate
            batch_norm(bool): using tf.keras.layers.BatchNormalization() or None
        """  
        super(BasicConv, self).__init__()

        kernel_regularizer = tf.keras.regularizers.L2(L2_reg) if L2_reg else None # Wer das liest ist blöd

        self.dropout_rate = dropout_rate
        if self.dropout_rate: # include dropout_rate
           self.dropout_layer = tf.keras.layers.Dropout(dropout_rate)
        
        # creating a list of layers
        self.layer_list = [tf.keras.layers.Conv2D(input_shape=(32, 32, 3), kernel_size=(2, 2), padding='same', strides=(2, 2), filters=32),
                           tf.keras.layers.Conv2D(input_shape=(32, 32, 3), kernel_size=(2, 2), padding='same', strides=(2, 2), filters=32),
                           tf.keras.layers.Conv2D(input_shape=(32, 32, 3), kernel_size=(2, 2), padding='same', strides=(2, 2), filters=32),
                           tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='same'),
                           tf.keras.layers.Conv2D(kernel_size=(2, 2), padding='same', strides=(2, 2), filters=64),
                           tf.keras.layers.Conv2D(kernel_size=(2, 2), padding='same', strides=(2, 2), filters=64),
                           tf.keras.layers.Conv2D(kernel_size=(2, 2), padding='same', strides=(2, 2), filters=64),
                           tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='same'),
                           tf.keras.layers.Flatten(),
                           tf.keras.layers.Dense(256, activation='relu'),
                           tf.keras.layers.Dense(128, activation='relu'),
                           tf.keras.layers.Dense(100, activation='softmax')]

                           

        if batch_norm: # if batch_norm, add tf.keras.layers.BatchNormalization() layers
          self.layer_list = [tf.keras.layers.Conv2D(32, 3, activation='relu', kernel_regularizer=kernel_regularizer),
                              tf.keras.layers.BatchNormalization(),
                              tf.keras.layers.Conv2D(32, 3,activation='relu',  kernel_regularizer=kernel_regularizer),
                              tf.keras.layers.BatchNormalization(),
                              tf.keras.layers.MaxPooling2D(pool_size=2, strides=2),
                              tf.keras.layers.BatchNormalization(),
                              tf.keras.layers.Conv2D(64, 3, activation='relu', kernel_regularizer=kernel_regularizer),
                              tf.keras.layers.BatchNormalization(),
                              tf.keras.layers.Conv2D(64, 3, activation='relu', kernel_regularizer=kernel_regularizer),
                              tf.keras.layers.BatchNormalization(),
                              tf.keras.layers.GlobalAvgPool2D(),
                              tf.keras.layers.BatchNormalization(),
                              tf.keras.layers.Dense(100, activation='softmax', kernel_regularizer=kernel_regularizer)]

        # old model layers
        '''
        self.convlayer1 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)),
        self.convlayer2 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)),
        self.pooling = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)
        self.convlayer3 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001))
        self.convlayer4 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001))
        self.global_pool = tf.keras.layers.GlobalAvgPool2D()
        self.out = tf.keras.layers.Dense(10, activation='softmax')
        '''

        # optimizer, loss function and metrics
        self.metrics_list = [tf.keras.metrics.Mean(name="loss"),
                             tf.keras.metrics.CategoricalAccuracy(name="acc")]
        

        self.loss_function = tf.keras.losses.CategoricalCrossentropy()
        self.optimizer = tf.keras.optimizers.SGD(learning_rate=0.005) # optimizer Adam

    def call(self, x, training=False):

        """
        activates the net and feeds information forward through layers
        also calculates loss and adjusts weights
        Args:
            x(tf.tensor): data for NN, input images with corresponding targets
            training (boolean) : indicating whether the layer should behave in training mode (adding dropout) or in inference mode (doing nothing)      
        Returns: output from NN                           
        """
        for layer in self.layer_list[:-1]:
                x = layer(x)
                if self.dropout_rate:
                    x = self.dropout_layer(x, training)
        return self.layer_list[-1](x)

    
    # 3. metrics property
    @property
    def metrics(self):
        return self.metrics_list
        # return a list with all metrics in the model

    # 4. reset all metrics objects
    def reset_metrics(self):
        """
        return a list with all metrics in the model
        """
        for metric in self.metrics:
            metric.reset_states()

    # added to get frobenius graph
    def compute_frobenius(self):
        """compute and return frobenius norm
        Returns:
            (tensor): frobenius norm
        """
        frobenius_norm = tf.zeros((1,))
        for var in self.trainable_variables:
            frobenius_norm += tf.norm(var, ord="euclidean")
        return frobenius_norm

    # 5. train step method
    def train_step(self, data):
        """
        training the network for once
        Args:
            data: input data (image with target)
        Returns:
            Return a dictionary mapping metric names to current value
        """

        img, label = data
        
        with tf.GradientTape() as tape:
            output = self(img, training=True)
            label = tf.reshape(label, output.shape)
            #loss = self.loss_function(label, output)
            loss = self.loss_function(label, output) + tf.reduce_sum(self.losses)
            
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        
        # update the state of the metrics according to loss and accuracy
        self.metrics[0].update_state(loss)
        self.metrics[1].update_state(label, output)

        # return a dictionary with metric names as keys and metric results as values
        return {m.name: m.result() for m in self.metrics}


    # 6. test_step method
    def test_step(self, data):

        """
        testing the network for once
        Args:
            data: input data (image with target)
        Returns:
            Return a dictionary mapping metric names to current value
        """

        img, label = data
        output = self(img, training=False)
        label = tf.reshape(label, output.shape)
        #loss = self.loss_function(label, output)
        loss = self.loss_function(label, output) + tf.reduce_sum(self.losses)

        # update the state of the metrics according to loss and accuracy
        self.metrics[0].update_state(loss)
        self.metrics[1].update_state(label, output)
        #print({m.name: m.result() for m in self.metrics})

        # return a dictionary with metric names as keys and metric results as values
        return {m.name: m.result() for m in self.metrics}

# CNN Model

In [ ]:
#from tensorflow.python.framework.importer import import_graph_def_for_function
class BasicConv(tf.keras.Model):
    def __init__(self, L2_reg=0, dropout_rate=0, batch_norm=False): # penalties, dropout, batch normalization

        """ 
        subclass of the tf.keras.Model class, creates metrics
        Args:
            L2_reg(float) = regularizer that applies a L2 regularization penalty
            dropout_rate(float) = sets input units to 0 with a frequency dropout_rate
            batch_norm(bool): using tf.keras.layers.BatchNormalization() or None
        """  
        super(BasicConv, self).__init__()

        kernel_regularizer = tf.keras.regularizers.L2(L2_reg) if L2_reg else None # Wer das liest ist blöd

        self.dropout_rate = dropout_rate
        if self.dropout_rate: # include dropout_rate
           self.dropout_layer = tf.keras.layers.Dropout(dropout_rate)
        
        # creating a list of layers
        self.layer_list = [tf.keras.layers.Conv2D(32, 3, activation='relu', kernel_regularizer=kernel_regularizer),
                           tf.keras.layers.Conv2D(32, 3,activation='relu',  kernel_regularizer=kernel_regularizer),
                           tf.keras.layers.MaxPooling2D(pool_size=2, strides=2),
                           tf.keras.layers.Conv2D(64, 3, activation='relu', kernel_regularizer=kernel_regularizer),
                           tf.keras.layers.Conv2D(64, 3, activation='relu', kernel_regularizer=kernel_regularizer),
                           tf.keras.layers.GlobalAvgPool2D(),
                           tf.keras.layers.Dense(100, activation='softmax', kernel_regularizer=kernel_regularizer)]

                           

        if batch_norm: # if batch_norm, add tf.keras.layers.BatchNormalization() layers
          self.layer_list = [tf.keras.layers.Conv2D(32, 3, activation='relu', kernel_regularizer=kernel_regularizer),
                              tf.keras.layers.BatchNormalization(),
                              tf.keras.layers.Conv2D(32, 3,activation='relu',  kernel_regularizer=kernel_regularizer),
                              tf.keras.layers.BatchNormalization(),
                              tf.keras.layers.MaxPooling2D(pool_size=2, strides=2),
                              tf.keras.layers.BatchNormalization(),
                              tf.keras.layers.Conv2D(64, 3, activation='relu', kernel_regularizer=kernel_regularizer),
                              tf.keras.layers.BatchNormalization(),
                              tf.keras.layers.Conv2D(64, 3, activation='relu', kernel_regularizer=kernel_regularizer),
                              tf.keras.layers.BatchNormalization(),
                              tf.keras.layers.GlobalAvgPool2D(),
                              tf.keras.layers.BatchNormalization(),
                              tf.keras.layers.Dense(100, activation='softmax', kernel_regularizer=kernel_regularizer)]

        # old model layers
        '''
        self.convlayer1 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)),
        self.convlayer2 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)),
        self.pooling = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)
        self.convlayer3 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001))
        self.convlayer4 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001))
        self.global_pool = tf.keras.layers.GlobalAvgPool2D()
        self.out = tf.keras.layers.Dense(10, activation='softmax')
        '''

        # optimizer, loss function and metrics
        self.metrics_list = [tf.keras.metrics.Mean(name="loss"),
                             tf.keras.metrics.CategoricalAccuracy(name="acc")]
        

        self.loss_function = tf.keras.losses.CategoricalCrossentropy()
        self.optimizer = tf.keras.optimizers.SGD(learning_rate=0.005) # optimizer Adam

    def call(self, x, training=False):

        """
        activates the net and feeds information forward through layers
        also calculates loss and adjusts weights
        Args:
            x(tf.tensor): data for NN, input images with corresponding targets
            training (boolean) : indicating whether the layer should behave in training mode (adding dropout) or in inference mode (doing nothing)      
        Returns: output from NN                           
        """
        for layer in self.layer_list[:-1]:
                x = layer(x)
                if self.dropout_rate:
                    x = self.dropout_layer(x, training)
        return self.layer_list[-1](x)

    
    # 3. metrics property
    @property
    def metrics(self):
        return self.metrics_list
        # return a list with all metrics in the model

    # 4. reset all metrics objects
    def reset_metrics(self):
        """
        return a list with all metrics in the model
        """
        for metric in self.metrics:
            metric.reset_states()

    # added to get frobenius graph
    def compute_frobenius(self):
        """compute and return frobenius norm
        Returns:
            (tensor): frobenius norm
        """
        frobenius_norm = tf.zeros((1,))
        for var in self.trainable_variables:
            frobenius_norm += tf.norm(var, ord="euclidean")
        return frobenius_norm

    # 5. train step method
    def train_step(self, data):
        """
        training the network for once
        Args:
            data: input data (image with target)
        Returns:
            Return a dictionary mapping metric names to current value
        """

        img, label = data
        
        with tf.GradientTape() as tape:
            output = self(img, training=True)
            label = tf.reshape(label, output.shape)
            #loss = self.loss_function(label, output)
            loss = self.loss_function(label, output) + tf.reduce_sum(self.losses)
            
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        
        # update the state of the metrics according to loss and accuracy
        self.metrics[0].update_state(loss)
        self.metrics[1].update_state(label, output)

        # return a dictionary with metric names as keys and metric results as values
        return {m.name: m.result() for m in self.metrics}


    # 6. test_step method
    def test_step(self, data):

        """
        testing the network for once
        Args:
            data: input data (image with target)
        Returns:
            Return a dictionary mapping metric names to current value
        """

        img, label = data
        output = self(img, training=False)
        label = tf.reshape(label, output.shape)
        #loss = self.loss_function(label, output)
        loss = self.loss_function(label, output) + tf.reduce_sum(self.losses)

        # update the state of the metrics according to loss and accuracy
        self.metrics[0].update_state(loss)
        self.metrics[1].update_state(label, output)
        #print({m.name: m.result() for m in self.metrics})

        # return a dictionary with metric names as keys and metric results as values
        return {m.name: m.result() for m in self.metrics}

# TRaining

In [42]:
mymodel = BasicConv()

In [ ]:
# TASK 4 - Training the networks

def training(mymodel,train_cifar, test_cifar, train_losses, test_losses, test_accuracies, train_accuracies):

  """
  trains a neural net for 15 epochs
  Args:
    train_losses(numpy.ndarray):  where we save train_losses for visualization
    test_losses(numpy.ndarray): save test_losses for visualization
    test_accuracies(numpy.ndarray): save train_accuracies for visualization
    train_accuracies(numpy.ndarray): save test_accuracies for visualization
  """
  
  # instantiate the model
  #mymodel = BasicConv()

  # test once in the beginning to check accuracy before training
  for data in test_cifar:
    metrics = mymodel.test_step(data)
          
  print("without training: ")
  print([f"test_{key}: {value.numpy()}" for (key, value) in metrics.items()], "\n")
    
  # reset all metrics
  mymodel.reset_metrics()  


  # internal training loop function
  def training_loop(model, train_cifar, test_cifar, epochs): 

    """
    inner training function, trains a neural net for n epochs
    Args:
      model(tf.keras.Model): model we want to use
      epochs(int): number of epochs we want to train the model
      train_ds(tf.tensor): preprocessed training data
      train_ds(tf.tensor): preprocessed testing data
    """

    for epoch in range(epochs):
        print(f"Epoch {epoch}:")
   

        # Training:
        for data in train_cifar:
            metrics = model.train_step(data)
            
        # print the metrics
        print([f"train_{key}: {value.numpy()}" for (key, value) in metrics.items()])

        for key, value in metrics.items():
            if key == "loss":
              train_losses.append(value.numpy())
            elif key == "acc":
              train_accuracies.append(value.numpy())

        # reset all metrics
        model.reset_metrics()
        

        # Testing:
        for data in test_cifar:
            metrics = model.test_step(data)
       
        print([f"test_{key}: {value.numpy()}" for (key, value) in metrics.items()])

        for key, value in metrics.items():
          if key == "loss":
            test_losses.append(value.numpy())
          elif key == "acc":
            test_accuracies.append(value.numpy())


        # reset all metrics
        model.reset_metrics() 
        print("\n")  

  
  training_loop(mymodel, train_cifar, test_cifar, 30) #epochs=15
  return train_losses, test_losses, test_accuracies, train_accuracies

train_losses = []
test_losses = []
train_accuracies = []
test_accuracies = []

train_losses, test_losses, test_accuracies, train_accuracies = training(mymodel,train_cifar, test_cifar, train_losses, test_losses, test_accuracies, train_accuracies)

without training: 
['test_loss: 4.60564661026001', 'test_acc: 0.009700000286102295'] 

Epoch 0:
['train_loss: 4.603296279907227', 'train_acc: 0.012140000239014626']
['test_loss: 4.600547790527344', 'test_acc: 0.011500000022351742']


Epoch 1:
['train_loss: 4.597024917602539', 'train_acc: 0.018239999189972878']
['test_loss: 4.592490196228027', 'test_acc: 0.01810000091791153']


Epoch 2:
['train_loss: 4.583359241485596', 'train_acc: 0.024299999698996544']
['test_loss: 4.570327281951904', 'test_acc: 0.02329999953508377']


Epoch 3:
['train_loss: 4.546092510223389', 'train_acc: 0.024739999324083328']
['test_loss: 4.5185627937316895', 'test_acc: 0.02969999983906746']


Epoch 4:
['train_loss: 4.46338415145874', 'train_acc: 0.041039999574422836']
['test_loss: 4.384319305419922', 'test_acc: 0.05119999870657921']


Epoch 5:
['train_loss: 4.2458577156066895', 'train_acc: 0.06313999742269516']
['test_loss: 4.134215354919434', 'test_acc: 0.06970000267028809']


Epoch 6:
['train_loss: 4.04108858108

In [ ]:
############# NO OPTIMIzations preventing overiftting ####################

def create_CNN_model():

    model = Sequential()
    model.add(Conv2D(input_shape=(32, 32, 3), kernel_size=(2, 2), padding='same', strides=(2, 2), filters=32))
    model.add(Conv2D(input_shape=(32, 32, 3), kernel_size=(2, 2), padding='same', strides=(2, 2), filters=32))
    model.add(Conv2D(input_shape=(32, 32, 3), kernel_size=(2, 2), padding='same', strides=(2, 2), filters=32))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='same'))

    model.add(Conv2D(kernel_size=(2, 2), padding='same', strides=(2, 2), filters=64))
    model.add(Conv2D(kernel_size=(2, 2), padding='same', strides=(2, 2), filters=64))
    model.add(Conv2D(kernel_size=(2, 2), padding='same', strides=(2, 2), filters=64))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='same'))

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(100, activation='softmax'))


    model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['categorical_accuracy'])

    return model



In [ ]:
########## CREATE MODEL ###########

cnnn_model = create_CNN_model()

cnnn_model.summary()

# ResNet50 Model

In [ ]:
############## ResNet50 MODEL  #############

resnet_model = tf.keras.applications.resnet.ResNet50(
       input_shape=(32,32,3), #.output_shapes()[0],
       include_top=True,
       weights=None)

resnet_model.compile(
       optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), #optimizer=tf.keras.optimizers.SGD(lr=0.001),
      loss='sparse_categorical_crossentropy', #loss='binary_crossentropy',
       metrics=['accuracy'])


# Train & Test Model
weights and tensorboard logging

In [ ]:
#set tensorboard 
# current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# logging_callback = tf.keras.callbacks.TensorBoard(log_dir=f"./logs/CNN_model/{current_time}")

# Train the model
cnnn_model.fit(cifar_data, batch_size=64, epochs=5) #callbacks=[logging_callback]) #batch_size=64, 

# Test Step
test_loss, test_acc = cnnn_model.evaluate(cifar_data)
print("test accuracy: ", test_acc)

# save the weights
# cnn_model.save_weights('./checkpoints/my_checkpoint')

# logging with tensorboard
# %tensorboard --logdir="logs/CNN_model"

In [ ]:
########### LOAD OLD WEIGHTS #########
'''
# Create a new model instance
new_model = create_CNN_model()

# Restore the weights
new_model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss, acc = new_model.evaluate(x_test_cifar, y_test_cifar)
print("test accuracy: ", acc)'''


# Confusion Matrix